# Домашнее задание по теме «Рекомендации на основе содержания»

```
- Использовать датасет MovieLens.
- Построить рекомендации (регрессия, предсказываем оценку) на фичах:
-   - TF-IDF на тегах и жанрах;
-   - средние оценки (+ median, variance и т. д.) пользователя и фильма.
- Оценить RMSE на тестовой выборке.
```

In [58]:
import pandas as pd
import duckdb
from  duckdb.typing import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor, KNeighborsClassifier

In [59]:
con = duckdb.connect()

In [60]:
def preprocess_text_for_tag(g: str):
    return g.replace(' ', '_').replace('-', '').lower()

con.create_function('preprocess_text_for_tag', preprocess_text_for_tag, [VARCHAR], VARCHAR)

In [ ]:
con.sql("CREATE OR REPLACE TABLE movies  AS (SELECT * FROM './data/ml-latest-small/movies.csv');")
con.sql("CREATE OR REPLACE TABLE tags    AS (SELECT * FROM './data/ml-latest-small/tags.csv');")
con.sql("CREATE OR REPLACE TABLE ratings AS (SELECT * FROM './data/ml-latest-small/ratings.csv');")
con.sql("CREATE OR REPLACE TABLE links   AS (SELECT * FROM './data/ml-latest-small/links.csv');")

In [ ]:
movies_df = con.sql("""
    WITH 
    genres_df as (
        SELECT movieId, string_agg(t.tags, ' ') tags 
        FROM (
            SELECT 
                movieId, 
                unnest(string_split(preprocess_text_for_tag(genres), '|')) tags
            FROM movies 
        ) t 
        GROUP BY ALL 
    ),
    tags_df as (
        SELECT movieId, string_agg(preprocess_text_for_tag(tag), ' ') tags FROM tags GROUP BY ALL
    )
    SELECT 
            movies.movieId
        ,   regexp_extract(movies.title, '(\d\d\d\d)') as year
        ,   movies.title
        ,   coalesce(genres_df.tags, '') genres
        ,   coalesce(tags_df.tags, '') tags
        ,   coalesce(count(r.rating), 0)                ratings
        ,   coalesce(min(r.rating), 0)                  min_rating
        ,   coalesce(max(r.rating) , 0)                 max_rating
        ,   coalesce(avg(r.rating), 0)                  avg_rating
        ,   coalesce(quantile_disc(r.rating, 0.5), 0)   mean_rating
        ,   coalesce(quantile_disc(r.rating, 0.9), 0)   rating_p90
        ,   coalesce(quantile_disc(r.rating, 0.95), 0)  rating_p95
        ,   coalesce(quantile_disc(r.rating, 0.99), 0)  rating_p99
    FROM movies
    LEFT JOIN genres_Df on movies.movieId = genres_df.movieId
    LEFT JOIN tags_df on movies.movieId = tags_df.movieId
    LEFT JOIN ratings r on r.movieId = movies.movieId
    GROUP BY ALL
""").df()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_238312/2030390583.py:1: SyntaxWarning: invalid escape sequence '\d'
  movies_df = con.sql("""


In [81]:
tags_tfidf = TfidfVectorizer()
tags_v = tags_tfidf.fit_transform(movies_df['tags'])
tags_v_df = pd.DataFrame(tags_v.toarray(), columns=tags_tfidf.get_feature_names_out())
movies_df = pd.concat([movies_df, tags_v_df], axis = 1)

In [82]:
genres_tfidf = TfidfVectorizer()
genres_v = genres_tfidf.fit_transform(movies_df['genres'])
genres_v_df = pd.DataFrame(genres_v.toarray(), columns=genres_tfidf.get_feature_names_out())
movies_df = pd.concat([movies_df, genres_v_df], axis = 1)

In [83]:
df = con.sql("""
    SELECT
        r.*,
        movies_df.*
    FROM ratings r
    LEFT JOIN movies_df on movies_df.movieId = r.movieId   
""").df()

In [84]:
X = con.sql(""" SELECT df.* EXCLUDE (timestamp, movieId_1, title, genres, tags, rating) FROM df """).df()
y = con.sql(""" SELECT rating FROM df """).df()

In [96]:
from sklearn.linear_model    import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics         import root_mean_squared_error

X = con.sql(""" SELECT df.* EXCLUDE (timestamp, movieId_1, title, genres, tags, rating) FROM df """).df()
y = con.sql(""" SELECT rating FROM df """).df()

X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=42, test_size=0.2 )

In [ ]:
reg = LinearRegression()
reg.fit( X_train, y_train )
print( root_mean_squared_error( y_train, reg.predict(X_train) ) )
print( root_mean_squared_error( y_test,  reg.predict(X_test) ) )

0.8752672394695049
0.8854531849220415
